Load packages

In [ ]:
rm(list = ls())
library(car)
library(rms)
library(pROC)
library(rmda)
library(PredictABEL)
library(reportROC)
library(predtools)
library(magrittr)
library(dplyr)
library(dcurves)
library(caret)
library(nomogramFormula)

Load data

In [ ]:
PC_origin <-read.csv('form/merge/pc.csv') 
PC <-read.csv('form/merge/pc_zscore.csv')
VC <-read.csv('form/merge/vc_zscore.csv')
TC1 <-read.csv('form/merge/tc1_zscore.csv') 
TC2 <-read.csv('form/merge/tc2_zscore.csv') 
dd <- datadist(PC)
old_options<- options(datadist = 'dd')

MODEL

In [ ]:
colnames(PC_origin) <- c("names", "label", "Age", "tuber", "Breast long", "Breast short", "Rad score", "DL score")
colnames(PC) <- c("names", "label", "Age", "tuber", "Breast long", "Breast short", "Rad score", "DL score")
colnames(VC) <- c("names", "label", "Age", "tuber", "Breast long", "Breast short", "Rad score", "DL score")
colnames(TC1) <- c("names", "label", "Age", "tuber", "Breast long", "Breast short", "Rad score", "DL score")
colnames(TC2) <- c("names", "label", "Age", "tuber", "Breast long", "Breast short", "Rad score", "DL score")

In [ ]:
DLRN <- lrm(label~Age + `Breast long` + `Breast short` + `Rad score` + `DL score`, data = PC, x = TRUE, y = TRUE, maxit = 1000)
DLRN

In [ ]:
Clinical <- lrm(label~Age + `Breast long` + `Breast short` + tuber, data = PC,x = TRUE, y = TRUE , maxit = 1000)
Radiomics <- lrm(label~`Rad score`, data = PC,x = TRUE, y = TRUE , maxit = 1000)
DL <- lrm(label~`DL score`, data = PC, x = TRUE, y = TRUE , maxit = 1000)

SAVE model predict prob

In [ ]:
dir.create('form/prob')

In [ ]:
DLRN_PC = predict(DLRN, type="fitted", newdata=PC)
Clinical_PC = predict(Clinical, type="fitted", newdata=PC)
Radiomics_PC = predict(Radiomics, type="fitted", newdata=PC)
DL_PC = predict(DL, type="fitted", newdata=PC)

DLRN_VC = predict(DLRN, type="fitted", newdata=VC)
Clinical_VC = predict(Clinical, type="fitted", newdata=VC)
Radiomics_VC = predict(Radiomics, type="fitted", newdata=VC)
DL_VC = predict(DL, type="fitted", newdata=VC)

DLRN_TC1 = predict(DLRN, type="fitted", newdata=TC1)
Clinical_TC1 = predict(Clinical, type="fitted", newdata=TC1)
Radiomics_TC1 = predict(Radiomics, type="fitted", newdata=TC1)
DL_TC1 = predict(DL, type="fitted", newdata=TC1)

DLRN_TC2 = predict(DLRN, type="fitted", newdata=TC2)
Clinical_TC2 = predict(Clinical, type="fitted", newdata=TC2)
Radiomics_TC2 = predict(Radiomics, type="fitted", newdata=TC2)
DL_TC2 = predict(DL, type="fitted", newdata=TC2)

df_tra <- data.frame(
    names = PC['names'],
    DLRN_PC = DLRN_PC,
    Clinical_PC = Clinical_PC,
    Radiomics_PC = Radiomics_PC,
    DL_PC = DL_PC
)

df_val <- data.frame(
    names = VC['names'],
    DLRN_VC = DLRN_VC,
    Clinical_VC = Clinical_VC,
    Radiomics_VC = Radiomics_VC,
    DL_VC = DL_VC
)

df_tc1 <- data.frame(
    names = TC1['names'],
    DLRN_TC1 = DLRN_TC1,
    Clinical_TC1 = Clinical_TC1,
    Radiomics_TC1 = Radiomics_TC1,
    DL_TC1 = DL_TC1
)

df_tc2 <- data.frame(
    names = TC2['names'],
    DLRN_TC2 = DLRN_TC2,
    Clinical_TC2 = Clinical_TC2,
    Radiomics_TC2 = Radiomics_TC2,
    DL_TC2 = DL_TC2
)

write.csv(df_tra, 'form/prob/pc.csv', row.names = FALSE)
write.csv(df_val, 'form/prob/vc.csv', row.names = FALSE)
write.csv(df_tc1, 'form/prob/tc1.csv', row.names = FALSE)
write.csv(df_tc2, 'form/prob/tc2.csv', row.names = FALSE)

### 95% CI for OR

In [ ]:
report_roc <- function(data, model){
    modelroc <- roc(data$label, predict(model, type='fitted', newdata=data))
    coords(modelroc, "best", ret=c("threshold", "specificity", "sensitivity", "accuracy"))
    
    detailROC <-reportROC(gold=data$label, predictor = predict(model, type='fitted', newdata=data))
    print(paste0('AUC(95%CI):',detailROC['AUC'],'(',paste(detailROC['AUC.low'],detailROC['AUC.up'],sep='-'),')'))
    print(paste0('ACC(95%CI):',detailROC['ACC'],'(',paste(detailROC['ACC.low'],detailROC['ACC.up'],sep='-'),')'))
    print(paste0('SEN(95%CI):',detailROC['SEN'],'(',paste(detailROC['SEN.low'],detailROC['SEN.up'],sep='-'),')'))
    print(paste0('SPE(95%CI):',detailROC['SPE'],'(',paste(detailROC['SPE.low'],detailROC['SPE.up'],sep='-'),')'))
}

In [ ]:
report_roc(PC, DLRN)
report_roc(VC, DLRN)
report_roc(TC1, DLRN)
report_roc(TC2, DLRN)

report_roc(PC, DL)
report_roc(VC, DL)
report_roc(TC1, DL)
report_roc(TC2, DL)

report_roc(PC, Radiomics)
report_roc(VC, Radiomics)
report_roc(TC1, Radiomics)
report_roc(TC2, Radiomics)

report_roc(PC, Clinical)
report_roc(VC, Clinical)
report_roc(TC1, Clinical)
report_roc(TC2, Clinical)

### nomogram

In [ ]:
dd <- datadist(PC)
old_options

In [ ]:
nom <- nomogram(DLRN ,fun= function(x)1/(1+exp(-x)), #逻辑回归公式
                fun.at = c(0.01,0.1,0.5,0.9,0.99),#风险轴刻度
                funlabel="Risk", #风险轴便签
                lp=F ,#是否显示系数轴
                conf.int = F
                # nint =4
                )
                # maxscale = 100
                # addrev=F)
nom

In [ ]:
dir.create('form/nom_score')

In [ ]:
options(old_options)


results <- formula_rd(nomogram = nom)

nom_score <- points_cal(formula = results$formula, rd = PC)
# boxplot(nom_score[PC$label == 0],nom_score[PC$label == 1],names = c("0","1"),col = colors()[10:11])
df <- data.frame(pred = points_cal(formula = results$formula, rd = PC), label = PC$label)
write.csv(df, 'form/nom_score/pc.csv',row.names = FALSE)

nom_score <- points_cal(formula = results$formula, rd = VC)
# boxplot(nom_score[VC$label == 0],nom_score[VC$label == 1],names = c("0","1"),col = colors()[10:11])
df <- data.frame(pred = points_cal(formula = results$formula, rd = VC), label = VC$label)
write.csv(df, 'form/nom_score/vc.csv',row.names = FALSE)

nom_score <- points_cal(formula = results$formula, rd = TC1)
# boxplot(nom_score[TC1$label == 0],nom_score[TC1$label == 1],names = c("0","1"),col = colors()[10:11])
df <- data.frame(pred = points_cal(formula = results$formula, rd = TC1), label = TC1$label)
write.csv(df, 'form/nom_score/tc1.csv',row.names = FALSE)

nom_score <- points_cal(formula = results$formula, rd = TC1)
# boxplot(nom_score[TC2$label == 0],nom_score[TC2$label == 1],names = c("0","1"),col = colors()[10:11])
df <- data.frame(pred = points_cal(formula = results$formula, rd = TC2), label = TC2$label)
write.csv(df, 'form/nom_score/tc2.csv',row.names = FALSE)

In [ ]:
#临床数据的诺模图数值映射
# for (i in colnames(PC_origin)) { 
#     if (i == "label") {next}
#     #nom[[i]][[i]] = round(nom[[i]][[i]]*sd(unlist(PC_souce[i]))+mean(unlist(PC_souce[i])))
#     #next
#     if (i == "Age" || i == "Breast long" || i == "Breast short") {
#         nom[[i]][[i]] = round(nom[[i]][[i]]*sd(unlist(PC_origin[i]))+mean(unlist(PC_origin[i])))
#         next
#     }
    # if (i == "Rad score" || i == "DL score"){
    #     nom[[i]][[i]] = round(nom[[i]][[i]]*sd(unlist(PC_origin[i]))+mean(unlist(PC_origin[i])))
    #     next
    # }
}

# pdf("plot/DLRN_nomogram.pdf", width = 10, height = 8,family="GB1") 
# plot(nom,
#      xfrac = 0.35,
#      cex.var=1.1,
#      cex.axis=1,
#      label.every=1,
#      tcl=-0.2,
#      lmgp = 0.5,
#      force.label=F
# )
# dev.off()
